# Analysis the jump and label

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy import optimize

import psutil
import os
import gc

In [ ]:
def read_files(route):
    file = open(f"{route}","r")
    info_r1 = file.readlines()
    file.close()
    
#     print("---------------------------------------------------------")
#     info = psutil.virtual_memory()
#     print(psutil.Process(os.getpid()).memory_info().rss)
#     print(info.total)
#     print(info.percent)
#     print(psutil.cpu_count())


    info_s1 = []
    for i in range (len(info_r1)):
        info_s1.append(info_r1[i].split())
    info_r1 = []
        
        
#     print("---------------------------------------------------------")
    info = psutil.virtual_memory()
#     print(psutil.Process(os.getpid()).memory_info().rss)
#     print(info.total)
    print(info.percent)
#     print(psutil.cpu_count())
    
    ##   0% --- to --- 50%
    Natom_s1 = int(info_s1[3][0])
    index_s1 = []
    for i in range (len(info_s1)):
        if info_s1[i] == ['ITEM:', 'TIMESTEP']:
            index_s1.append(i)

    splited_trj1 = []
    for j in range (len(index_s1)):
        frame = np.zeros((Natom_s1,5))
        for i in range (Natom_s1):
            frame[i,0] = int(info_s1[index_s1[j]+i+9][0])
            frame[i,1] = float(info_s1[index_s1[j]+i+9][1])
            frame[i,2] = float(info_s1[index_s1[j]+i+9][2])
            frame[i,3] = float(info_s1[index_s1[j]+i+9][3])
            frame[i,4] = float(info_s1[index_s1[j]+i+9][4])
        splited_trj1.append([frame,info_s1[index_s1[j]+5:index_s1[j]+8]])
    
    return splited_trj1

In [ ]:
def distance(pos1,pos2,bx,by,bz):
    
    vec = np.zeros((1,3))
    vec[0,0] = pos2[0]-pos1[0]
    vec[0,1] = pos2[1]-pos1[1]
    vec[0,2] = pos2[2]-pos1[2]
    
    return np.sqrt(vec[0,0]**2+vec[0,1]**2+vec[0,2]**2)

In [ ]:
def get_average_distance_soft(traj_name,target):
    ## which sequence
    
    traj = copy.deepcopy(traj_name)
    boundary_x = float(traj[target][1][0][1])-float(traj[target][1][0][0])
    boundary_y = float(traj[target][1][1][1])-float(traj[target][1][1][0])
    boundary_z = float(traj[target][1][2][1])-float(traj[target][1][2][0])

    ## distribution
    O_index = []
    for i in range (0,len(traj[target][0])):
        if traj[target][0][i][1] == 15.999:
            O_index.append(i)

    O_O_distance = []
    for chain_ in range (chain_numbers):
        for position_ in range (1,d_of_p):
            dis = distance(traj[target][0][O_index[chain_*3*d_of_p+position_*3]][2:],traj[target][0][O_index[chain_*3*d_of_p+position_*3-1]][2:],boundary_x,boundary_y,boundary_z)
            O_O_distance.append(dis)
    
    return O_O_distance

In [ ]:
def get_average_distance_rigid(traj_name,target):
    ## which sequence
    
    traj = copy.deepcopy(traj_name)
    boundary_x = float(traj[target][1][0][1])-float(traj[target][1][0][0])
    boundary_y = float(traj[target][1][1][1])-float(traj[target][1][1][0])
    boundary_z = float(traj[target][1][2][1])-float(traj[target][1][2][0])

    ## distribution
    O_index = []
    for i in range (0,len(traj[target][0])):
        if traj[target][0][i][1] == 15.999:
            O_index.append(i)

    O_O_distance = []
    for chain_ in range (chain_numbers):
        for position_ in range (0,d_of_p):
            dis = distance(traj[target][0][O_index[chain_*3*d_of_p+position_*3]][2:],traj[target][0][O_index[chain_*3*d_of_p+position_*3+2]][2:],boundary_x,boundary_y,boundary_z)
            O_O_distance.append(dis)

    return O_O_distance

In [ ]:
def extract(route,gather_soft,gather_rigid):
    
    splited_trj1 = read_files(f"stretch{temperature}/{route}")
    for i in range (len(splited_trj1)-1):
        gather_soft.append(get_average_distance_soft(splited_trj1,i))
        gather_rigid.append(get_average_distance_rigid(splited_trj1,i))

    return gather_soft,gather_rigid

# For D

In [ ]:
name="pdkcd"
temperature = "1.2"
sname = "d"

chain_numbers = 48
d_of_p = 32 # degree of polymerization

In [ ]:
gather_soft_pre = []
gather_rigid_pre = []

gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0_ta-1.trj",gather_soft_pre,gather_rigid_pre)
gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0_ta-2.trj",gather_soft_pre,gather_rigid_pre)

In [ ]:
averaged_L0_soft = np.zeros((len(gather_soft_pre[0]),2))
averaged_L0_rigid = np.zeros((len(gather_rigid_pre[0]),2))

for id_ in range (len(gather_soft_pre[0])):
    averaged_L0_soft[id_,0] = id_
    
    for t in range (len(gather_soft_pre)):
        averaged_L0_soft[id_,1] += gather_soft_pre[t][id_]
    averaged_L0_soft[id_,1] = averaged_L0_soft[id_,1]/len(gather_soft_pre)

for id_ in range (len(gather_rigid_pre[0])):
    averaged_L0_rigid[id_,0] = id_
    
    for t in range (len(gather_rigid_pre)):
        averaged_L0_rigid[id_,1] += gather_rigid_pre[t][id_]
    averaged_L0_rigid[id_,1] = averaged_L0_rigid[id_,1]/len(gather_rigid_pre)


In [ ]:
 averaged_L0_soft,averaged_L0_rigid

In [ ]:
gather_soft = []
gather_rigid = []

gather_soft,gather_rigid = extract(f"{sname}/s1.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s2.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s3.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s4.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s5.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s6.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s7.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s8.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s9.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s10.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s11.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s12.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s13.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s14.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s15.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s16.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s17.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s18.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s19.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s20.trj",gather_soft,gather_rigid)

In [ ]:
length_revolution_d_soft = np.zeros((len(gather_soft),2))
length_revolution_d_rigid = np.zeros((len(gather_rigid),2))

for t in range (len(gather_soft)):
    length_revolution_d_soft[t,0] = 0.05*t
    length_revolution_d_rigid[t,0] = 0.05*t
    
    for id_ in range (len(gather_soft[0])):
        length_revolution_d_soft[t,1] += gather_soft[t][id_]/averaged_L0_soft[id_,1]
    length_revolution_d_soft[t,1] = length_revolution_d_soft[t,1]/len(gather_soft[0])
    
    for id_ in range (len(gather_rigid[0])):
        length_revolution_d_rigid[t,1] += gather_rigid[t][id_]/averaged_L0_rigid[id_,1]
    length_revolution_d_rigid[t,1] = length_revolution_d_rigid[t,1]/len(gather_rigid[0])

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(length_revolution_d_soft[:,0],length_revolution_d_soft[:,1]/length_revolution_d_soft[2,1],label="soft")
ax.scatter(length_revolution_d_rigid[:,0],length_revolution_d_rigid[:,1]/length_revolution_d_rigid[0,1],label="rigid")
# ax.set_xlabel("Strain (%)",size = 20)
ax.set_xlabel("Time (ns)",size = 20)
ax.set_ylabel(f"$ <L/ L_{0}>$",size = 20)
ax.set_title(f"{name}",size = 20)
ax.set_ylim(0.8,1.2)
ax.set_ylim(0.99,1.11)

plt.legend(loc="upper right",fontsize=20)
plt.show()

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Strain"+"     ")
file.write("<L/L0>(soft)"+"     ")
file.write("<L/L0>(rigid)"+"     ")

file.write("\n")

for i in range (len(length_revolution_d_soft)):
    file.write("%.2f"%(length_revolution_d_soft[i,0])+"    ")
    file.write("%.5f"%(length_revolution_d_soft[i,1])+" ")
    file.write("%.5f"%(length_revolution_d_rigid[i,1])+" ")
    file.write("\n")
    
file.close()

## for M

In [ ]:
name="pdkcm"
temperature = "1.2"
sname = "m"

chain_numbers =44
d_of_p = 43 # degree of polymerization

In [ ]:
gather_soft_pre = []
gather_rigid_pre = []

gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0-ta.trj",gather_soft_pre,gather_rigid_pre)

In [ ]:
averaged_L0_soft = np.zeros((len(gather_soft_pre[0]),2))
averaged_L0_rigid = np.zeros((len(gather_rigid_pre[0]),2))

for id_ in range (len(gather_soft_pre[0])):
    averaged_L0_soft[id_,0] = id_
    
    for t in range (len(gather_soft_pre)):
        averaged_L0_soft[id_,1] += gather_soft_pre[t][id_]
    averaged_L0_soft[id_,1] = averaged_L0_soft[id_,1]/len(gather_soft_pre)

for id_ in range (len(gather_rigid_pre[0])):
    averaged_L0_rigid[id_,0] = id_
    
    for t in range (len(gather_rigid_pre)):
        averaged_L0_rigid[id_,1] += gather_rigid_pre[t][id_]
    averaged_L0_rigid[id_,1] = averaged_L0_rigid[id_,1]/len(gather_rigid_pre)


In [ ]:
gather_soft = []
gather_rigid = []

gather_soft,gather_rigid = extract(f"{sname}/s1.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s2.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s3.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s4.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s5.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s6.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s7.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s8.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s9.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s10.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s11.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s12.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s13.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s14.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s15.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s16.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s17.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s18.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s19.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s20.trj",gather_soft,gather_rigid)

In [ ]:
length_revolution_m_soft = np.zeros((len(gather_soft),2))
length_revolution_m_rigid = np.zeros((len(gather_rigid),2))

for t in range (len(gather_soft)):
    length_revolution_m_soft[t,0] = 0.05*t
    length_revolution_m_rigid[t,0] = 0.05*t
    
    for id_ in range (len(gather_soft[0])):
        length_revolution_m_soft[t,1] += gather_soft[t][id_]/averaged_L0_soft[id_,1]
    length_revolution_m_soft[t,1] = length_revolution_m_soft[t,1]/len(gather_soft[0])
    
    for id_ in range (len(gather_rigid[0])):
        length_revolution_m_rigid[t,1] += gather_rigid[t][id_]/averaged_L0_rigid[id_,1]
    length_revolution_m_rigid[t,1] = length_revolution_m_rigid[t,1]/len(gather_rigid[0])

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(length_revolution_m_soft[:,0],length_revolution_m_soft[:,1]/length_revolution_m_soft[0,1],label="soft")
ax.scatter(length_revolution_m_rigid[:,0],length_revolution_m_rigid[:,1]/length_revolution_m_rigid[0,1],label="rigid")
ax.set_xlabel("Time (ns)",size = 20)
ax.set_ylabel(f"$ <L/ L_{0}>$",size = 20)
ax.set_title(f"{name}",size = 20)
ax.set_ylim(0.8,1.2)
ax.set_ylim(0.99,1.11)

plt.legend(loc="upper right",fontsize=20)
plt.savefig(f"{name}_O_O_ave.pdf")
plt.show()

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Strain"+"     ")
file.write("<L/L0>(soft)"+"     ")
file.write("<L/L0>(rigid)"+"     ")

file.write("\n")

for i in range (len(length_revolution_m_soft)):
    file.write("%.2f"%(length_revolution_m_soft[i,0])+"    ")
    file.write("%.5f"%(length_revolution_m_soft[i,1])+" ")
    file.write("%.5f"%(length_revolution_m_rigid[i,1])+" ")
    file.write("\n")
    
file.close()

## for P

In [ ]:
name="pdkcp"
temperature = "1.2"
sname = "p"

chain_numbers = 38
d_of_p = 48 # degree of polymerization

In [ ]:
gather_soft_pre = []
gather_rigid_pre = []

gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0_ta.trj",gather_soft_pre,gather_rigid_pre)

In [ ]:
averaged_L0_soft = np.zeros((len(gather_soft_pre[0]),2))
averaged_L0_rigid = np.zeros((len(gather_rigid_pre[0]),2))

for id_ in range (len(gather_soft_pre[0])):
    averaged_L0_soft[id_,0] = id_
    
    for t in range (len(gather_soft_pre)):
        averaged_L0_soft[id_,1] += gather_soft_pre[t][id_]
    averaged_L0_soft[id_,1] = averaged_L0_soft[id_,1]/len(gather_soft_pre)

for id_ in range (len(gather_rigid_pre[0])):
    averaged_L0_rigid[id_,0] = id_
    
    for t in range (len(gather_rigid_pre)):
        averaged_L0_rigid[id_,1] += gather_rigid_pre[t][id_]
    averaged_L0_rigid[id_,1] = averaged_L0_rigid[id_,1]/len(gather_rigid_pre)


In [ ]:
gather_soft = []
gather_rigid = []

gather_soft,gather_rigid = extract(f"{sname}/s1.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s2.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s3.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s4.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s5.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s6.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s7.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s8.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s9.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s10.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s11.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s12.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s13.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s14.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s15.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s16.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s17.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s18.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s19.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s20.trj",gather_soft,gather_rigid)

In [ ]:
length_revolution_p_soft = np.zeros((len(gather_soft),2))
length_revolution_p_rigid = np.zeros((len(gather_rigid),2))

for t in range (len(gather_soft)):
    length_revolution_p_soft[t,0] = 0.05*t
    length_revolution_p_rigid[t,0] = 0.05*t
    
    for id_ in range (len(gather_soft[0])):
        length_revolution_p_soft[t,1] += gather_soft[t][id_]/averaged_L0_soft[id_,1]
    length_revolution_p_soft[t,1] = length_revolution_p_soft[t,1]/len(gather_soft[0])
    
    for id_ in range (len(gather_rigid[0])):
        length_revolution_p_rigid[t,1] += gather_rigid[t][id_]/averaged_L0_rigid[id_,1]
    length_revolution_p_rigid[t,1] = length_revolution_p_rigid[t,1]/len(gather_rigid[0])

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(length_revolution_p_soft[:,0],length_revolution_p_soft[:,1]/length_revolution_p_soft[0,1],label="soft")
ax.scatter(length_revolution_p_rigid[:,0],length_revolution_p_rigid[:,1]/length_revolution_p_rigid[0,1],label="rigid")
ax.set_xlabel("Time (ns)",size = 20)
ax.set_ylabel(f"$ <L/ L_{0}>$",size = 20)
ax.set_title(f"{name}",size = 20)
ax.set_ylim(0.8,1.2)
ax.set_ylim(0.99,1.11)

plt.legend(loc="upper right",fontsize=20)
plt.savefig(f"{name}_O_O_ave.pdf")
plt.show()

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Strain"+"     ")
file.write("<L/L0>(soft)"+"     ")
file.write("<L/L0>(rigid)"+"     ")

file.write("\n")

for i in range (len(length_revolution_p_soft)):
    file.write("%.2f"%(length_revolution_p_soft[i,0])+"    ")
    file.write("%.5f"%(length_revolution_p_soft[i,1])+" ")
    file.write("%.5f"%(length_revolution_p_rigid[i,1])+" ")
    file.write("\n")
    
file.close()

## for H

In [ ]:
name="pdkch"
temperature = "1.2"
sname = "h"

chain_numbers = 46
d_of_p =  36 # degree of polymerization

In [ ]:
gather_soft_pre = []
gather_rigid_pre = []

gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0_ta.trj",gather_soft_pre,gather_rigid_pre)
gather_soft_pre,gather_rigid_pre = extract(f"{sname}/s0_ta-2.trj",gather_soft_pre,gather_rigid_pre)

In [ ]:
averaged_L0_soft = np.zeros((len(gather_soft_pre[0]),2))
averaged_L0_rigid = np.zeros((len(gather_rigid_pre[0]),2))

for id_ in range (len(gather_soft_pre[0])):
    averaged_L0_soft[id_,0] = id_
    
    for t in range (len(gather_soft_pre)):
        averaged_L0_soft[id_,1] += gather_soft_pre[t][id_]
    averaged_L0_soft[id_,1] = averaged_L0_soft[id_,1]/len(gather_soft_pre)

for id_ in range (len(gather_rigid_pre[0])):
    averaged_L0_rigid[id_,0] = id_
    
    for t in range (len(gather_rigid_pre)):
        averaged_L0_rigid[id_,1] += gather_rigid_pre[t][id_]
    averaged_L0_rigid[id_,1] = averaged_L0_rigid[id_,1]/len(gather_rigid_pre)


In [ ]:
gather_soft = []
gather_rigid = []

gather_soft,gather_rigid = extract(f"{sname}/s1.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s2.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s3.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s4.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s5.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s6.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s7.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s8.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s9.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s10.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s11.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s12.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s13.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s14.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s15.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s16.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s17.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s18.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s19.trj",gather_soft,gather_rigid)
gather_soft,gather_rigid = extract(f"{sname}/s20.trj",gather_soft,gather_rigid)

In [ ]:
length_revolution_h_soft = np.zeros((len(gather_soft),2))
length_revolution_h_rigid = np.zeros((len(gather_rigid),2))

for t in range (len(gather_soft)):
    length_revolution_h_soft[t,0] = 0.05*t
    length_revolution_h_rigid[t,0] = 0.05*t
    
    for id_ in range (len(gather_soft[0])):
        length_revolution_h_soft[t,1] += gather_soft[t][id_]/averaged_L0_soft[id_,1]
    length_revolution_h_soft[t,1] = length_revolution_h_soft[t,1]/len(gather_soft[0])
    
    for id_ in range (len(gather_rigid[0])):
        length_revolution_h_rigid[t,1] += gather_rigid[t][id_]/averaged_L0_rigid[id_,1]
    length_revolution_h_rigid[t,1] = length_revolution_h_rigid[t,1]/len(gather_rigid[0])

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(length_revolution_h_soft[:,0],length_revolution_h_soft[:,1]/length_revolution_h_soft[0,1],label="soft")
ax.scatter(length_revolution_h_rigid[:,0],length_revolution_h_rigid[:,1]/length_revolution_h_rigid[0,1],label="rigid")
ax.set_xlabel("Time (ns)",size = 20)
ax.set_ylabel(f"$ <L/ L_{0}>$",size = 20)
ax.set_title(f"{name}",size = 20)
ax.set_ylim(0.8,1.2)
ax.set_ylim(0.99,1.11)

plt.legend(loc="upper right",fontsize=20)
plt.savefig(f"{name}_O_O_ave.pdf")
plt.show()

In [ ]:
file = open(f"{name}_{temperature}_data.txt","w")

file.write("Strain"+"     ")
file.write("<L/L0>(soft)"+"     ")
file.write("<L/L0>(rigid)"+"     ")

file.write("\n")

for i in range (len(length_revolution_h_soft)):
    file.write("%.2f"%(length_revolution_h_soft[i,0])+"    ")
    file.write("%.5f"%(length_revolution_h_soft[i,1])+" ")
    file.write("%.5f"%(length_revolution_h_rigid[i,1])+" ")
    file.write("\n")
    
file.close()

## Comparison

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)

ax.scatter(length_revolution_m[:,0],length_revolution_m[:,1],label="$S_{M}$",color="none", edgecolor="black",s=20)
ax.scatter(length_revolution_p_soft[:,0],length_revolution_p_soft[:,1]/length_revolution_p_soft[0,1],label="$S_{P}$",color="none", edgecolor="grey",s=20)
ax.scatter(length_revolution_h_soft[:,0],length_revolution_h_soft[:,1]/length_revolution_h_soft[0,1],label="$S_{H}$",color="none", edgecolor="red",s=20)
ax.scatter(length_revolution_d[:,0],length_revolution_d[:,1],label="$S_{D}$",color="none", edgecolor="blue",s=20)
ax.set_xlabel("Time (ns)",size = 20)
ax.set_ylabel(f"$ <L/ L_{0}>$",size = 20)
ax.set_ylim(0.8,1.2)
ax.set_ylim(0.99,1.11)

plt.legend(loc="upper left",fontsize=14)
plt.show()